# Figures present in the paper:

   - 1- figure of the proof of bounding truncated force from $A^{(0,1)}$ 
   - 2- Simulations figures homogeneous case 

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
plt.rc('axes', labelsize=22)
plt.rc('legend', fontsize=16)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# to delete this cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

# 1-figure of the proof of bounding truncated force from $A^{(0,1)}$

In [ ]:
from structure_factor.spatial_windows import BallWindow
def sphere_number(p,q, d):
    r = (p-q)/2
    window = BallWindow(center= [0]*d, radius=q + r)
    return window.surface/(2*r**(d-1))
def next_points(a_0, p, q):
    r = (p-q)/2
    x_1 = (q**2-r**2 + 2*q*r)/(q+r)
    return np.array([[x_1, np.sqrt((q+r)**2 - x_1**2)], [x_1, - np.sqrt((q+r)**2 - x_1**2)]])
def next_points_2(a, p, q):
    r = (p-q)/2
    b_ = (4*r**2*a[1]**2 - 2*a[1]*(q+r)**2)
    a_ = a[1]**2 + a[0]**2 
    c_ = (q + r)**4 + 4*r**4 - 4*r**2*(q + r)**2 - a[0]**2*(q+r)**2
    delta = b_**2 - 4*a_*c_
    print(delta)
    x_2 = [(-b_-np.sqrt(delta))/(2*a_), (-b_+np.sqrt(delta))/(2*a_)]
    print(x_2)
    x_1 = [((q+r)**2 - 2*r**2 - x_2[0]*a[1])/a[1], ((q+r)**2 - 2*r**2 - x_2[1]*a[1])/a[1]]
    return np.array([[x_1[0], x_2[0]], [x_1[1], x_2[1]]]) 

In [ ]:
import numpy as np

epsilon = 0.5
x = np.array([3,76])
a_0 = [76, 0]
d=2
origin = [0]*d
norm_x = np.linalg.norm(x)
R=8
r=R/epsilon
p, q = norm_x + r, norm_x - r
sphere_nb = sphere_number(p=p, q=q, d=d)
points_next = next_points(a_0, p, q)
print(np.linalg.norm(points_next, axis=1))


In [ ]:

print("sphere number=", sphere_nb)
window0 = BallWindow(center=origin, radius=norm_x)
window1 = BallWindow(center=origin, radius=R)
window2 = BallWindow(center=origin, radius=norm_x-R/epsilon)
window3 = BallWindow(center=origin, radius=norm_x + R/epsilon)
window4 = BallWindow(center=x, radius=R/epsilon)
window5 = BallWindow(center=[np.sqrt((q+r)**2 - (q/(q+r))**2), q/(q+r)], radius=r)
window6 = BallWindow(center=points_next[0,:], radius=r)
window7 = BallWindow(center=points_next[1,:], radius=r)

In [ ]:
import numpy as np
from structure_factor.spatial_windows import BallWindow

theta = np.arcsin(r/norm_x)
alpha = np.linspace(-theta, theta, 100)
u, v = (R+5)*np.cos(alpha), (R+5)*np.sin(alpha)
nb_theta = int(180/(theta*57.2958)/2)
print(nb_theta)

fig, ax = plt.subplots(figsize=(4,4))
window3.plot(axis=ax, color="b", linestyle="--")
window2.plot(axis=ax, color="b", linestyle="--")
ax.plot(0, 0, "k.")
plt.plot(a_0[0], a_0[1], 'r.')
plt.fill_between([norm_x-r,np.cos(theta)*(norm_x+r)], 
                 [np.sin(theta)*(norm_x-r), np.sin(theta)*(norm_x+r)], 
                 [-np.sin(theta)*(norm_x-r), -np.sin(theta)*(norm_x+r)], facecolor='green', interpolate=True)
y_ = np.linspace(-np.sin(theta)*(norm_x-r), np.sin(theta)*(norm_x-r))
plt.fill_betweenx(y_, np.sqrt((norm_x -r)**2 - y_**2), np.sqrt((norm_x + r)**2 - y_**2),
                  facecolor='green', interpolate=True)
y_ = np.linspace(-np.sin(theta)*(norm_x), np.sin(theta)*(norm_x))
plt.fill_betweenx(y_, np.sqrt((norm_x)**2 - y_**2), np.sqrt((norm_x + r)**2 - y_**2),
                  facecolor='green', interpolate=True)
for i in range(0, nb_theta ):
    a,b= np.cos(2*i*theta)*norm_x, np.sin(2*i*theta)*norm_x
    window = BallWindow(center=[a,b ], radius=r)
    window.plot(axis=ax, color="k")
    plt.plot(a, b, 'r.')
    window = BallWindow(center=[a,-b ], radius=r)
    window.plot(axis=ax, color="k")
    plt.plot(a, -b, 'r.')
    plt.plot([np.cos((2*i+1)*theta)*(norm_x-r), np.cos((2*i+1)*theta)*(norm_x+r)], 
             [np.sin((2*i+1)*theta)*(norm_x-r), np.sin((2*i+1)*theta)*(norm_x+r)], 'b' )
    plt.plot([np.cos((2*i+1)*theta)*(norm_x-r), np.cos((2*i+1)*theta)*(norm_x+r)], 
             [-np.sin((2*i+1)*theta)*(norm_x-r), -np.sin((2*i+1)*theta)*(norm_x+r)], 'b' )
c, d = np.cos(2*7*theta)*norm_x, np.sin(2*7*theta)*norm_x
window = BallWindow(center=[c,d ], radius=r)
window.plot(axis=ax, color="k")
plt.plot(c, d, 'r.')
plt.plot([0, c], [0, d], 'grey', linestyle="--")
plt.gca().annotate(r'$\|\mathbf{x}\|_2$', xy=(c/2, d -5), xycoords='data', fontsize=8, rotation=-10)
plt.plot([0,  np.cos(theta)*(norm_x+r)], 
        [0, np.sin(theta)*(norm_x+r)], 'b' )
plt.plot([0, np.cos(theta)*(norm_x+r)], 
        [0, -np.sin(theta)*(norm_x+r)], 'b' )
plt.plot([np.cos(15*theta)*(norm_x-r), np.cos(15*theta)*(norm_x+r)], 
             [np.sin((15)*theta)*(norm_x-r), np.sin((15)*theta)*(norm_x+r)], 'b' )
plt.plot([norm_x-r,norm_x],[0, 0], 'k' )
plt.gca().annotate(r'$f(\mathbf{x})$', xy=(norm_x -15, 3), xycoords='data', fontsize=8)
plt.plot([0,-8],[0, 0], 'k' )
plt.gca().annotate(r'$R$', xy=(-6, -6), xycoords='data', fontsize=7)
window1.plot(axis=ax, color="k")
window0.plot(axis=ax, color="grey", linestyle="-")
plt.plot(u,v, "grey")
plt.gca().annotate(r'$\theta$', xy=(R+7, -2), xycoords='data', fontsize=8)
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
plt.savefig("proof_existance_pic.pdf")
plt.show()


# 2-  Simulation figures homogeneous case

In [ ]:
# Creat a poisson point process
import matplotlib.pyplot as plt
import numpy as np
import statistics as stat
import time 

from structure_factor.point_processes import HomogeneousPoissonPointProcess
from structure_factor.point_pattern import PointPattern
from structure_factor.spatial_windows import BallWindow, BoxWindow
from GPPY.gravity_point_process import GravityPointProcess

## d=2

In [ ]:
d = 2
r = 10
q, p=0, 4

simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 200
points_nb = intensity*simu_window.volume 
print("points_nb=", points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window, seed=0) 
points=poisson_pp.points

gpp_pp = GravityPointProcess(poisson_pp)

epsilon_critical = gpp_pp.epsilon_critical
print("critical epsilon=", epsilon_critical)
start = time.time()
push_pp_critical = gpp_pp.pushed_point_pattern(epsilon=epsilon_critical, stop_time=1)
end = time.time()- start
print("time=", end % 60)
push_pp_critical_trunc = gpp_pp.pushed_point_pattern(epsilon=epsilon_critical, 
                                               p=p, q=q, correction=False
                                               ) 


In [ ]:
s = r-p
l = s/(np.sqrt(2))
obs_window = BoxWindow([[-l/2, l/2]]*d)

fig, axis = plt.subplots(1, 3, figsize=(10, 5))
poisson_pp.plot(window=obs_window, axis=axis[0], c="b", s=0.2)
push_pp_critical.plot(window=obs_window, axis=axis[1], c="k", s=0.2)
axis[1].set_title(r"Push using $F$")
push_pp_critical_trunc.plot(window=obs_window, axis=axis[2], c="k", s=0.2)
axis[2].set_title(r"Push using $\tilde{F}$")
axis[0].set_title("Poisson")
plt.savefig("poisson_and_push_2D.pdf")
plt.show()

# d=3

In [ ]:
d = 3
r=10
simu_window = BallWindow(center=[0]*d, radius=r)
intensity = 20
points_nb = intensity*simu_window.volume 
print("estimate points_nb=", points_nb)
# list of poisson pp
poisson = HomogeneousPoissonPointProcess(intensity)
poisson_pp = poisson.generate_point_pattern(window=simu_window) 
points=poisson_pp.points
fig = plt.figure(figsize=(4, 4))
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.scatter3D(points[:, 0], points[:, 1], points[:, 2], c=points[:, 2], s=0.2, cmap='Greens')
ax.view_init(40, 60)
plt.show()

In [ ]:
q, p=0, 4
gpp_pp = GravityPointProcess(poisson_pp)

epsilon_critical = gpp_pp.epsilon_critical
print("critical epsilon=", epsilon_critical)
start = time.time()
push_pp_critical = gpp_pp.pushed_point_pattern(epsilon=epsilon_critical, stop_time=1, 
                                               core_number=6)
end_1 = time.time()- start
print("time=", int(end_1/60), "min ", end_1%60, "s")
start_2 = time.time()
push_pp_critical_trunc = gpp_pp.pushed_point_pattern(epsilon=epsilon_critical, 
                                                     p=p, q=q, correction=False,
                                                     core_number=1)
end_2 = time.time()- start_2
print("time=", int(end_2/60), "min ", end_2%60, "s")

In [ ]:
r=10
s = r-p
l = s/(np.sqrt(2))
obs_window = BoxWindow([[-l/2, l/2]]*d)

push_points = push_pp_critical.restrict_to_window(obs_window).points
push_points_trunc = push_pp_critical_trunc.restrict_to_window(obs_window).points
points = poisson_pp.restrict_to_window(obs_window).points

fig = plt.figure(figsize=(15, 6))
ax = fig.add_subplot(1, 3, 1, projection='3d')
ax.scatter3D(points[:, 0], points[:, 1], points[:, 2], c=points[:, 2], s=1, cmap='Blues')
ax.view_init(40, 60)
ax.set_title(r"Poisson")
ax = fig.add_subplot(1, 3, 2, projection='3d')
ax.scatter3D(push_points[:, 0], push_points[:, 1], push_points[:, 2], 
             c=push_points[:, 2], s=1, cmap='Greys')
ax.view_init(40, 60)
ax.set_title(r"Push using $F$")
ax = fig.add_subplot(1, 3, 3, projection='3d')
ax.scatter3D(push_points_trunc[:, 0], push_points_trunc[:, 1], push_points_trunc[:, 2], 
             c=push_points_trunc[:, 2], s=1, cmap='Greys')
ax.set_title(r"Push using $\tilde{F}$")
ax.view_init(40, 60)
plt.savefig("poisson_and_push_3D.pdf")
plt.show()